In [1]:
import spacy

In [2]:
import pytextrank

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('./datasets/patent.tsv',sep='\t')

/Users/ifti/anaconda3/envs/pytextrank/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df

,id,type,number,country,date,abstract,title,kind,num_claims,filename,withdrawn
0,10000000,utility,10000000,US,2018-06-19,A frequency modulated (coherent) laser detecti...,Coherent LADAR using intra-pixel quadrature de...,B2,20,ipg180619.xml,0
1,10000001,utility,10000001,US,2018-06-19,The injection molding machine includes a fixed...,Injection molding machine and mold thickness c...,B2,12,ipg180619.xml,0
2,10000002,utility,10000002,US,2018-06-19,The present invention relates to: a method for...,Method for manufacturing polymer film and co-e...,B2,9,ipg180619.xml,0
3,10000003,utility,10000003,US,2018-06-19,The invention relates to a method for producin...,Method for producing a container from a thermo...,B2,18,ipg180619.xml,0
4,10000004,utility,10000004,US,2018-06-19,The present invention relates to provides a do...,"Process of obtaining a double-oriented film, c...",B2,6,ipg180619.xml,0
...,...,...,...,...,...,...,...,...,...,...,...
7814191,T998013,defensive publication,T998013,US,1980-09-02,NaN,Protection of insect pheromones from degradati...,I4,1,pftaps19800902_wk36.zip,0
7814192,T998014,defensive publication,T998014,US,1980-09-02,NaN,"Thiazolyl couplers, coupler compositions and p...",I4,3,pftaps19800902_wk36.zip,0
7814193,T999001,defensive publication,T999001,US,1980-10-07,NaN,Sack handling device,I4,1,pftaps19801007_wk41.zip,0
7814194,T999002,defensive publication,T999002,US,1980-10-07,NaN,Application of polymeric powders to a substrate,I4,7,pftaps19801007_wk41.zip,0


### Cleanup
Patents date > 2010
Abstract is not null
Patent is not withdrawn

In [6]:
df_gt_2010 = df[df['date'].str[:4].astype(int) >= 2010]
df_gt_2010[df_gt_2010['abstract'].notnull()]
df_gt_2010 = df_gt_2010[df_gt_2010['withdrawn']==0]

In [7]:
df_gt_2020 = df[df['date'].str[:4].astype(int) >= 2020]
df_gt_2020[df_gt_2020['abstract'].notnull()]
df_gt_2020 = df_gt_2020[df_gt_2020['withdrawn']==0]

In [8]:
df_gt_2021 = df[df['date'].str[:4].astype(int) >= 2021]
df_gt_2021[df_gt_2021['abstract'].notnull()]
df_gt_2021 = df_gt_2021[df_gt_2021['withdrawn']==0]

In [9]:
df_gt_2021.count()

id            186075
type          186075
number        186075
country       186075
date          186075
abstract      168788
title         186075
kind          186075
num_claims    186075
filename      186075
withdrawn     186075
dtype: int64

In [30]:
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
def top_topics(text, max_items=5):
    # load a spaCy model, depending on language, scale, etc.
   
    doc = nlp(str(text))

    # examine the top-ranked phrases in the document
    top_ranked = []
    count = 0
    for phrase in doc._.phrases:
        count += 1
        if phrase.rank > .1 and count <= max_items:
#             print(phrase.text)
#             print(phrase.rank, phrase.count)
#             print(phrase.chunks)
            top_ranked.append(phrase.text)
        else:
            break

        
    return top_ranked

In [14]:
df_gt_2021.count()

id            186075
type          186075
number        186075
country       186075
date          186075
abstract      168788
title         186075
kind          186075
num_claims    186075
filename      186075
withdrawn     186075
dtype: int64

In [19]:
df_1000 = df_gt_2021[:1000]

In [23]:
%%time
df_1000['top_topics'] = df_1000.apply(lambda row: top_topics(row['abstract']), axis=1)

CPU times: user 20 s, sys: 79.5 ms, total: 20 s
Wall time: 20 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [27]:
l=['a','a','b','c', 1]
set(l)

{1, 'a', 'b', 'c'}

In [31]:
%%time
df_gt_2021['top_topics'] = df_gt_2021.apply(lambda row: top_topics(row['abstract']), axis=1)

CPU times: user 58min 6s, sys: 4.81 s, total: 58min 11s
Wall time: 1h 20min 4s


In [33]:
df_gt_2021.to_csv('patents_topics_gt_2021.csv')

## Merge with Location and assignee

In [34]:
df_assignee = pd.read_csv('./datasets/assignee.tsv',sep='\t')
df_patent_assignee = pd.read_csv('./datasets/patent_assignee.tsv', sep='\t')
df_location =  pd.read_csv('./datasets/location.tsv', sep='\t')
df_patent_assignee = df_patent_assignee.merge(df_assignee, left_on='assignee_id', right_on='id', suffixes=('_x', '_y'))
df_patent_assignee = df_patent_assignee.merge(df_location, left_on='location_id', right_on='id', suffixes=('', '_location'))

/Users/ifti/anaconda3/envs/pytextrank/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
df_gt_2021_assignee_location = df_gt_2021.merge(df_patent_assignee, left_on='id', right_on='patent_id', suffixes=('','_x'))

In [39]:
print(df_gt_2021_assignee_location.shape)
print(df_gt_2021.shape)

(180806, 29)
(186075, 12)


In [55]:
# df_gt_2021_assignee_location.drop(columns=['filename', 'state_fips', 'county_fips','latitude','longitude'])
df_gt_2021_assignee_location.drop(columns=['filename', 'state_fips', 'county_fips','latitude','longitude','id_x', 'withdrawn','kind','type_x','name_first','name_last','type','patent_id','location_id','assignee_id'])

,id,number,country,date,abstract,title,num_claims,top_topics,organization,id_location,city,state,country_x,county
0,10881042,10881042,US,2021-01-05,An agricultural implement having a rolling bas...,Agricultural implement with a scraper internal...,103,"[rotation, a rolling basket, the rolling basket]","Summers Manufacturing Company, Inc.",6b463091-cb8e-11eb-9615-121df0c29c1e,Devils Lake,ND,US,Ramsey
1,10881043,10881043,US,2021-01-05,A method for increasing the load on driving re...,Method for increasing the load on driving rear...,1,[],MENDELOVA UNIVERZITA V BRNE. AGRONOMICKA FAKUL...,3cb80671-cb8e-11eb-9615-121df0c29c1e,NaN,NaN,NaN,NaN
2,10881044,10881044,US,2021-01-05,An agricultural machine includes a frame havin...,Planter with full tandem offset pivot,17,[travel],AGCO Corporation,60ac7fc3-09bc-11ec-893a-12de62d610b1,Duluth,GA,US,Gwinnett
3,10881045,10881045,US,2021-01-05,A precision agriculture prescription system wh...,System and method for prescribing fertilizer a...,24,"[budgeted fertilizer, fertilizer application r...",NUTRIEN AG SOLUTIONS (CANADA) INC.,ff74c1c9-cb8f-11eb-9615-121df0c29c1e,Calgary,NaN,CA,NaN
4,10881046,10881046,US,2021-01-05,The present invention relates to a system that...,Agricultural liquid fertilizer and chemical de...,22,"[agricultural operations, tillage]","Agri-Inject, Inc.",dccbb60c-cb8e-11eb-9615-121df0c29c1e,Yuma,CO,US,Yuma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180801,RE48614,RE48614,US,2021-06-29,"In one embodiment, a method includes receiving...",Dynamic adjustment of received signal threshol...,43,"[signals, part]","Wacom Co., Ltd.",ff804f14-cb90-11eb-9615-121df0c29c1e,Saitama,NaN,JP,NaN
180802,RE48615,RE48615,US,2021-06-29,Methods and apparatus for use in communicating...,User interface methods and apparatus for use i...,20,"[message body contents, messages, conversation...",BlackBerry Limited,ff94740d-cb8f-11eb-9615-121df0c29c1e,Ontario,NaN,CA,NaN
180803,RE48616,RE48616,US,2021-06-29,A semiconductor structure includes a silicon-o...,Isolation region fabrication for replacement g...,13,"[active devices, a top silicon layer, the top ...","Samsung Electronics Co., Ltd.",ff9566ad-cb8f-11eb-9615-121df0c29c1e,Suwon-si,NaN,KR,NaN
180804,RE48617,RE48617,US,2021-06-29,Provided is a package of a light emitting diod...,Package of light emitting diode and method for...,19,[diode chip],"LG INNOTEK CO., LTD.",ff7e35db-cb90-11eb-9615-121df0c29c1e,Seoul,NaN,KR,NaN


In [85]:
headers = ['id','number','title','date','country','top_topics','assignee_id','location_id', 'organization','state','country_x']
df_gt_2021_assignee_location[:100].to_csv('patents_topics_gt_2021_100.csv', columns=headers, index=False)

In [80]:
df_gt_2021_assignee_location.count()

id              180806
type            180806
number          180806
country         180806
date            180806
abstract        167476
title           180806
kind            180806
num_claims      180806
filename        180806
withdrawn       180806
top_topics      180806
patent_id       180806
assignee_id     180806
location_id     180806
id_x            180806
type_x          180803
name_first        1316
name_last         1316
organization    179561
id_location     180806
city            176687
state            85362
country_x       176779
latitude        175130
longitude       175130
county           80328
state_fips       85362
county_fips      80328
dtype: int64